In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.svm import SVC # Support Vector Classifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split # train_test_split nếu bạn muốn tải lại
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler # Tùy chọn, để thử nghiệm scaling
import time # Để đo thời gian



In [2]:
# --- 0. (TÙY CHỌN) TẢI LẠI DỮ LIỆU VÀ LABEL ENCODER NẾU Ở NOTEBOOK MỚI ---
# Nếu bạn đang ở notebook mới, hãy bỏ comment phần này và chạy
print("Đang tải dữ liệu đã tiền xử lý...") # Bỏ comment dòng này nếu bạn muốn nó chạy
model_dir = '../models/'
processed_data_dir = '../data/processed/' # Sửa lại ở đây

# Các dòng sau đó giữ nguyên
X_train_df = pd.read_csv(os.path.join(processed_data_dir, 'X_train_filtered.csv'))
X_test_df = pd.read_csv(os.path.join(processed_data_dir, 'X_test_filtered.csv'))
y_train_series = pd.read_csv(os.path.join(processed_data_dir, 'y_train_filtered.csv'))['Disease_encoded']
y_test_series = pd.read_csv(os.path.join(processed_data_dir, 'y_test_filtered.csv'))['Disease_encoded']
label_encoder_filtered = joblib.load(os.path.join(model_dir, 'disease_label_encoder_filtered.pkl'))

X_train = X_train_df.values
X_test = X_test_df.values
y_train = y_train_series.values
y_test = y_test_series.values
print("Đã tải xong dữ liệu.")
print(f"Số lượng lớp trong label_encoder_filtered: {len(label_encoder_filtered.classes_)}")


Đang tải dữ liệu đã tiền xử lý...
Đã tải xong dữ liệu.
Số lượng lớp trong label_encoder_filtered: 754


In [3]:
# --- 1. KHỞI TẠO MODEL SVM NGUYÊN THỦY ---
print("\nKhởi tạo model SVM nguyên thủy...")

# Lựa chọn siêu tham số thủ công hoặc dùng mặc định
# Kernel 'rbf' thường là một lựa chọn tốt để bắt đầu cho các bài toán phức tạp.
# C = 1.0 là giá trị mặc định cho regularization.
# gamma = 'scale' là giá trị mặc định, tính bằng 1 / (n_features * X.var()).
# class_weight='balanced' vẫn rất quan trọng.
# probability=False để huấn luyện nhanh hơn. Nếu bạn cần xác suất, đặt là True
# nhưng thời gian huấn luyện sẽ tăng đáng kể.
svm_primitive_model = SVC(
    C=1.0,                 # Giá trị C mặc định, bạn có thể thử thay đổi (ví dụ: 0.1, 10)
    kernel='rbf',          # Thử 'rbf' trước, hoặc 'linear'
    gamma='scale',         # Mặc định cho RBF, bạn có thể thử 'auto' hoặc một giá trị số cụ thể
    class_weight='balanced',
    random_state=42,
    probability=False      # Đặt False để chạy nhanh
)


Khởi tạo model SVM nguyên thủy...


In [4]:
# --- 2. HUẤN LUYỆN MODEL SVM ---
print("\nBắt đầu huấn luyện model SVM nguyên thủy...")
print("Quá trình này VẪN CÓ THỂ MẤT THỜI GIAN ĐÁNG KỂ với dataset lớn của bạn, dù không dùngSearchCV.")

start_time_svm_primitive = time.time()

svm_primitive_model.fit(X_train, y_train) # Dùng X_train, y_train không scale (hoặc X_train_scaled nếu bạn đã scale)

end_time_svm_primitive = time.time()
print(f"\nHoàn thành huấn luyện SVM nguyên thủy sau: {((end_time_svm_primitive - start_time_svm_primitive) / 60):.2f} phút")



Bắt đầu huấn luyện model SVM nguyên thủy...
Quá trình này VẪN CÓ THỂ MẤT THỜI GIAN ĐÁNG KỂ với dataset lớn của bạn, dù không dùngSearchCV.

Hoàn thành huấn luyện SVM nguyên thủy sau: 22.20 phút


In [5]:
# --- 3. ĐÁNH GIÁ MODEL SVM NGUYÊN THỦY TRÊN TẬP TEST ---
print("\n--- Đánh giá Model SVM Nguyên thủy trên Tập Test ---")
y_pred_svm_primitive = svm_primitive_model.predict(X_test) # Dùng X_test (hoặc X_test_scaled)

accuracy_svm_primitive = accuracy_score(y_test, y_pred_svm_primitive)
print(f"Accuracy của Model SVM Nguyên thủy: {accuracy_svm_primitive:.4f}")

print("\nClassification Report của Model SVM Nguyên thủy:")
report_svm_primitive = classification_report(
    y_test,
    y_pred_svm_primitive,
    labels=np.arange(len(label_encoder_filtered.classes_)),
    target_names=label_encoder_filtered.classes_,
    zero_division=0
)
print(report_svm_primitive)


--- Đánh giá Model SVM Nguyên thủy trên Tập Test ---
Accuracy của Model SVM Nguyên thủy: 0.8634

Classification Report của Model SVM Nguyên thủy:
                                                          precision    recall  f1-score   support

                               abdominal aortic aneurysm       0.93      1.00      0.97        28
                                        abdominal hernia       0.95      0.99      0.97        81
                                         abscess of nose       0.66      0.93      0.77        58
                                     abscess of the lung       0.50      1.00      0.67         4
                                  abscess of the pharynx       0.89      0.93      0.91        68
                                    acanthosis nigricans       0.62      0.83      0.71         6
                                               acariasis       1.00      0.86      0.92         7
                                               achalasia       0.49 

In [7]:
# --- 4. LƯU MODEL SVM NGUYÊN THỦY ---
model_dir = '../models/' # Đảm bảo thư mục này tồn tại
# os.makedirs(model_dir, exist_ok=True) # Nếu chưa tạo

joblib.dump(svm_primitive_model, os.path.join(model_dir, 'svm_primitive_model.pkl'))
print(f"\nModel SVM nguyên thủy được lưu vào {os.path.join(model_dir, 'svm_primitive_model.pkl')}")


Model SVM nguyên thủy được lưu vào ../models/svm_primitive_model.pkl
